In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from tensorflow import keras
import os,sys
import albumentations as albu
from PIL import Image

%load_ext autoreload
%autoreload 2

# Loading the dataset

In [2]:
import helper

In [3]:
# Loaded a set of images
root_dir = "Datasets/training/"

n = 100

image_dir = root_dir + "images/"
files = os.listdir(image_dir)
print("Loading " + str(n) + " images")
imgs = [helper.load_image(image_dir + files[i]) for i in range(n)]

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " images")
gt_imgs = [helper.load_image(gt_dir + files[i]) for i in range(n)]

Loading 100 images
Loading 100 images


#### Data Augmentation
##### From https://towardsdatascience.com/road-detection-using-segmentation-models-and-albumentations-libraries-on-keras-d5434eaf73a8


In [4]:
#Doubles the nb of images. Needs improvement
for i in range(4):
    for n in range(len(gt_imgs)):
        augmented = helper.image_augmentation(image_size = 256)(image=imgs[n], mask=gt_imgs[n])
        imgs.append(augmented['image'])
        gt_imgs.append(augmented['mask'])

In [5]:
# Extract patches from input images
patch_size = 16 # each patch is 16*16 pixels

img_patches = [helper.img_crop(imgs[i], patch_size, patch_size) for i in range(len(imgs))]
gt_patches = [helper.img_crop(gt_imgs[i], patch_size, patch_size) for i in range(len(gt_imgs))]

# Linearize list of patches
img_patches = np.asarray([img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))])
gt_patches =  np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])
gt_patches = helper.patches_labelization(gt_patches)

## Feature processing 

#### Balancing amount of patches  background vs road for training

In [6]:

#img_patches, gt_patches = helper.feature_balancing(img_patches, gt_patches)
#Works badly!

# CNN

In [7]:
from tensorflow.keras import datasets, layers, models

## Layers

In [8]:
from keras.regularizers import l2
"""model = models.Sequential()
reg = 1e-6 # L2 regularization factor (used on weights, but not biases)
### heavily(!) inspired from a previous project. Needs to be changed/optimized
model.add(layers.Convolution2D(64, 5, 5, padding='same',input_shape=(16, 16, 3)))
model.add(layers.LeakyReLU(alpha=0.1))
model.add(layers.MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(layers.Dropout(0.25))
model.add(layers.Convolution2D(128, 3, 3,padding='same'))
model.add(layers.LeakyReLU(alpha=0.1))
model.add(layers.MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(layers.Dropout(0.25))
model.add(layers.Convolution2D(256, 3, 3, padding='same'))
model.add(layers.LeakyReLU(alpha=0.1))
model.add(layers.MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(layers.Dropout(0.25))
model.add(layers.Convolution2D(256, 3, 3, padding='same'))
model.add(layers.LeakyReLU(alpha=0.1))
model.add(layers.MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(layers.Dropout(0.25))
model.add(layers.Flatten())
model.add(layers.Dense(128, kernel_regularizer=l2(reg))) # Fully connected layer (128 neurons)
model.add(layers.LeakyReLU(alpha=0.1))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, kernel_regularizer=l2(reg)))
"""

"""model = models.Sequential()

model.add(layers.Conv2D(32, (2, 2), input_shape=(16, 16, 3)))
model.add(layers.LeakyReLU(alpha=0.1))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (2, 2)))
model.add(layers.LeakyReLU(alpha=0.1))
model.add(layers.Conv2D(128, (2, 2)))
model.add(layers.LeakyReLU(alpha=0.1))

model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64))
model.add(layers.LeakyReLU(alpha=0.1))
model.add(layers.Dense(2, activation='softmax'))"""
model = models.Sequential()
kernel_size = (2,2)
pool_size = (3,3)
alpha_relu = 0.1
regularizer = 1e-6
shape = (16, 16, 3)

model.add(layers.Convolution2D(64,kernel_size, input_shape=shape))
model.add(layers.LeakyReLU(alpha_relu))
model.add(layers.MaxPooling2D(pool_size))
model.add(layers.Convolution2D(64, kernel_size))
model.add(layers.Dropout(0.1))
model.add(layers.LeakyReLU(alpha_relu))

model.add(layers.Convolution2D(128, kernel_size))
model.add(layers.LeakyReLU(alpha_relu))
model.add(layers.MaxPooling2D(pool_size))
model.add(layers.Convolution2D(128,kernel_size))
model.add(layers.Dropout(0.1))
model.add(layers.LeakyReLU(alpha_relu))
model.add(layers.MaxPooling2D(pool_size))

model.add(layers.Convolution2D(256,kernel_size))
model.add(layers.LeakyReLU(alpha_relu))
model.add(layers.MaxPooling2D(pool_size))
model.add(layers.Convolution2D(256,kernel_size))
model.add(layers.Dropout(0.1))
model.add(layers.LeakyReLU(alpha_relu))

model.add(layers.Flatten())

model.add(layers.Dense(2))
model.add(layers.Activation('softmax'))

Using TensorFlow backend.


In [9]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 15, 15, 64)        832       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 15, 15, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 6, 6, 64)          16448     
_________________________________________________________________
dropout (Dropout)            (None, 6, 6, 64)          0         
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 5, 5, 128)         3

## Training

In [10]:

def generate_minibatch(X, Y, batch_size):
    """
    Procedure for real-time minibatch creation and image augmentation.
     This runs in a parallel thread while the model is being trained.
    """
    Y = tf.keras.utils.to_categorical(Y, 2)
    while 1:
        # Generate one minibatch
        X_batch = np.empty((batch_size, 16,16, 3))
        Y_batch = np.empty((batch_size, 2))
        for i in range(batch_size):
            # Select a random image
            idx = np.random.choice(X.shape[0])
            shape = X[idx].shape
                    
            # The label does not depend on the image rotation/flip (provided that the rotation is in steps of 90°)
            X_batch[i] = X[idx]
            Y_batch[i] = Y[idx]
        yield (X_batch, Y_batch)
          

In [ ]:
import math
batch_size = 64
steps_per_epoch = math.ceil(img_patches.shape[0] / batch_size)
"""def softmax_categorical_crossentropy(y_true, y_pred):
       
        Uses categorical cross-entropy from logits in order to improve numerical stability.
        This is especially useful for TensorFlow (less useful for Theano).           
        return K.categorical_crossentropy(y_pred, y_true, from_logits=True)
    """
model.compile(loss = "categorical_crossentropy",  optimizer = keras.optimizers.Adam(lr=1e-3),
              metrics = ['accuracy'])
#model.fit(img_patches, gt_patches, batch_size = 32, epochs = 2, validation_split = 0.2)
model.fit_generator(generate_minibatch(img_patches,gt_patches, batch_size),
                    epochs=15, steps_per_epoch= steps_per_epoch)


Epoch 1/15
15625/15625 [==============================] - 1515s 97ms/step - loss: 0.4643 - accuracy: 0.7749
Epoch 2/15
15625/15625 [==============================] - 1600s 102ms/step - loss: 0.4166 - accuracy: 0.8062
Epoch 3/15
15625/15625 [==============================] - 1988s 127ms/step - loss: 0.3921 - accuracy: 0.8203
Epoch 4/15
15625/15625 [==============================] - 2443s 156ms/step - loss: 0.3756 - accuracy: 0.83000s - loss: 0.37 - ETA: 5s - loss: 0.3
Epoch 5/15
15625/15625 [==============================] - 3139s 201ms/step - loss: 0.3634 - accuracy: 0.8368ET - ETA: 36:07 - loss: 0.3679  - ETA: 36:04 - loss: 0.36 - ETA: 35:50 - loss: - ETA: 35:05 - loss: 0.3679 - accura - ETA: 35:03 - loss: 0.3679 - accuracy: 0. - ETA: 35:03 - loss: 0.367 - ETA: 34:23 - loss: 0.3675 - - ETA: 34:20 - loss: 0 - ETA: 34:15 - loss: 0.3675 - accuracy - ETA: 34:14 - loss: 0.3675 - accuracy: 0.8 - ETA: 34:13 - loss: 0.3676 - accur - ETA: 34:12 - loss: 0.3 - ETA: 34:07 - loss:  - ETA: 33:56 - 

## Predictions

In [ ]:
#predictions = model.predict(img_patches)
#predictions = (predictions[:,0] < predictions[:,1]) * 1
#predictions = np.squeeze(predictions)

In [ ]:
#predictions = np.squeeze(helper.binarize_predictions(predictions))

In [ ]:
#np.set_printoptions(threshold=sys.maxsize)
#print(predictions)

# Displaying predictions

In [ ]:
"""# Display prediction as an image
img_idx = 2

img_indexed = imgs[img_idx]
img_pat = helper.img_crop(img_indexed, patch_size, patch_size)
img_pat = np.asarray(img_pat)
prediction = model.predict(img_pat)

w = gt_imgs[img_idx].shape[0]
h = gt_imgs[img_idx].shape[1]
predicted_im = helper.label_to_img_array(w, h, patch_size, patch_size, helper.binarize_predictions(prediction))
cimg = helper.concatenate_images(imgs[img_idx], predicted_im)
fig1 = plt.figure(figsize=(10, 10)) # create a figure with the default size 
plt.imshow(cimg, cmap='Greys_r')

new_img = helper.make_img_overlay(imgs[img_idx], predicted_im)

#plt.imshow(new_img)"""

# Create Submission

In [ ]:
import submission as sub

In [ ]:
n = 50
patch_size = 16 

img_test = []

for i in range(1, 51):
    image_filename = 'Datasets/test_set_images/test_' + str(i) + '/test_' + str(i) + '.png' 
    img_test.append(helper.load_image(image_filename))
img_patches_test = [helper.img_crop(img_test[i], patch_size, patch_size) for i in range(n)]
img_patches_test = np.asarray([img_patches_test[i][j] for i in range(len(img_patches_test))
                               for j in range(len(img_patches_test[i]))])

In [ ]:
predictions = model.predict(img_patches_test)
#predictions = np.squeeze(helper.binarize_predictions(predictions))
print(predictions.shape)
#print(predictions[10])


In [ ]:


np.set_printoptions(threshold=sys.maxsize)
print(predictions.shape)
print(predictions)


### Post-processing

In [ ]:

prediction_patches =[]
#predictions = np.squeeze(helper.binarize_predictions(predictions))
predictions = (predictions[:,0] < predictions[:,1]) * 1
predictions = np.squeeze(predictions)
for i in range(50):
    #predictions = np.squeeze(helper.binarize_predictions(predictions))
    #prediction = helper.binarize_predictions_array(predictions[i*1444 :((i*1444)+1444)])
    prediction = helper.label_to_img_patches_test(predictions[i*1444 :((i*1444)+1444)])
    prediction = helper.combine_surounded_patches(prediction) #Which order? Is it even useful. 
    prediction = helper.remove_lonely_patches(prediction)
    prediction_patches.append(prediction) 


In [ ]:

img_patches_test_mod = [helper.img_crop(prediction_patches[i], 1, 1)
                        for i in range(len(prediction_patches))]

img_patches_test_mod =  np.asarray([img_patches_test_mod[i][j] for i in range(len(img_patches_test_mod))
                                     for j in range(len(img_patches_test_mod[i]))])

"""img_patches_test_mod = np.asarray([prediction_patches[i][j] for i in range(len(prediction_patches)) 
                               for j in range(len(prediction_patches[i]))])"""

predictions_mod = np.squeeze(img_patches_test_mod)

sub.create_pred_images(predictions_mod)


In [ ]:
submission_filename = 'submission1.csv'
   
sub.create_submission(submission_filename)

In [ ]:
#Just to have a look..
print(img_patches_test_mod.shape)
print(img_patches_test.shape)
for i in range(25):
    fig1 = plt.figure(figsize=(5, 5))
    plt.imshow(prediction_patches[i], cmap='Greys_r')

In [ ]:
for i in range(25):
    fig1 = plt.figure(figsize=(5, 5))
    plt.imshow(img_test[i])